In [1]:
import os
import nest_asyncio  
nest_asyncio.apply()

from typing import List, Tuple
from llama_parse import LlamaParse
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

DATA_FOLDER = "data"

parser = LlamaParse(
    api_key="llx-3QORP75OUx11inHUpIy67FLzIgYc0gjfAGKRLDiECXOXkkne",
    result_type="markdown",
    num_workers=1,
    verbose=True,
    language="en",
)

def load_text_files(folder_path: str) -> str:
    all_text = ""
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                all_text += f.read() + "\n"
    return all_text

def create_vector_database(text_data: str, data_dir="faiss_data") -> Tuple:
    os.makedirs(data_dir, exist_ok=True)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=256
    )
    chunks = text_splitter.split_text(text_data)
    documents = [Document(page_content=chunk) for chunk in chunks]
    embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    vector_store = FAISS.from_documents(
        documents=documents, 
        embedding=embed_model
    )

    faiss_index_path = os.path.join(data_dir, "faiss_index")
    vector_store.save_local(faiss_index_path)

    return vector_store, embed_model
text_data = load_text_files(DATA_FOLDER)

vector_db, embed_model = create_vector_database(text_data)


ModuleNotFoundError: No module named 'llama_parse'

In [ ]:
import os
from typing import Tuple, List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

def dataframes_to_text(document) -> str:
    all_text = ""
    for doc in document:
        if hasattr(doc, 'text_resource') and doc.text_resource:
            all_text += doc.text_resource.text + "\n"
    return all_text

def create_vector_database(document, data_dir="faiss_data") -> Tuple:
    os.makedirs(data_dir, exist_ok=True)
    all_text = dataframes_to_text(document)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=64
    )
    chunks = text_splitter.split_text(all_text)
    documents = [Document(page_content=chunk) for chunk in chunks]
    embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    vector_store = FAISS.from_documents(
        documents=documents, 
        embedding=embed_model
    )

    faiss_index_path = os.path.join(data_dir, "faiss_index")
    vector_store.save_local(faiss_index_path)
    
    return vector_store, embed_model
vector_db, embed_model = create_vector_database(document)

C:\Users\kongl\AppData\Local\Temp\ipykernel_15304\297736213.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
c:\Users\kongl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh",
    openai_api_base="https://api.opentyphoon.ai/v1",
    model_name="typhoon-v2-70b-instruct",
    temperature=0.7,
    max_tokens=1024
)

def summarize_text(text: str, max_tokens: int = 512) -> str:
    tokens = text.split()
    return ' '.join(tokens[:max_tokens]) + '...' if len(tokens) > max_tokens else text

def generate_response(prompt: str) -> str:
    chat_completion = llm.client.chat.completions.create(
        model="typhoon-v2-70b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for the Bank of Thailand's Nano Finance Regulation."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,
        temperature=0.7,
        stop=None
    )
    return chat_completion.choices[0].message.content.strip()

def create_chatbot(vector_db):
    retriever = vector_db.as_retriever(search_kwargs={'k': 3})
    template = """
    You are an expert assistant tasked with providing detailed, accurate,
    and well-contextualized answers based solely on the provided context.
    Use the following context from the input documents or vector database to answer the question. 
    Do not rely on external knowledge unless explicitly stated otherwise. 
    Ensure your response is comprehensive, directly addresses the question, 
    and includes relevant details or explanations derived from the context. 
    If the context does not contain sufficient information to fully answer the question, 
    clearly state what is missing and provide the best possible response based on what is available.

    Context: {context}
    
    Question: {question}
    
    Answer:
    """
    
    PROMPT = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    return qa_chain

def ask_question(qa_chain: RetrievalQA, question: str) -> str:
    response = qa_chain({"query": question})
    return response['result']

qa_chain = create_chatbot(vector_db)

def chat_loop():
    print("Bot: Hello! I'm your Nano Finance Regulation assistant. Ask me anything!")
    print("(Type 'quit' to exit)")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit']:
            print("Bot: Goodbye!")
            break
            
        response = ask_question(qa_chain, user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    chat_loop()

Bot: Hello! I'm your Nano Finance Regulation assistant. Ask me anything!
(Type 'quit' to exit)
Bot: สรุปเกี่ยวกับ Data Management Concerns จากข้อมูลที่มีอยู่:

1. **การวางแผน**: การวางแผนที่ดีสำหรับการจัดการข้อมูลเป็นสิ่งสำคัญ เพื่อให้แน่ใจว่าข้อมูลถูกจัดเก็บและเข้าถึงได้อย่างเหมาะสม
2. **การควบคุม**: การควบคุมข้อมูลเพื่อป้องกันการสูญหาย การเข้าถึงที่ไม่ได้รับอนุญาต หรือการใช้งานข้อมูลในทางที่ผิด
3. **การจัดส่ง**: การจัดส่งข้อมูลให้กับผู้ใช้ที่ต้องการอย่างมีประสิทธิภาพ
4. **คุณภาพข้อมูล**: การรักษาคุณภาพของข้อมูลให้ดี เพื่อให้ข้อมูลที่ถูกต้องและเชื่อถือได้
5. **ความปลอดภัยข้อมูล**: การปกป้องข้อมูลจากการเข้าถึงที่ไม่ได้รับอนุญาตหรือการโจมตี
6. **เมตาดาต้า**: การจัดการเมตาดาต้าเพื่อให้สามารถเข้าใจและใช้ข้อมูลได้อย่างมีประสิทธิภาพ
7. **ข้อมูลอ้างอิง**: การจัดการข้อมูลอ้างอิงเพื่อให้มั่นใจว่าข้อมูลที่ใช้เป็นมาตรฐานเดียวกัน
8. **แพลตฟอร์มการกำกับดูแลข้อมูล**: การใช้แพลตฟอร์มที่เหมาะสมในการกำกับดูแลข้อมูล
9. **นโยบาย**: การพัฒนานโยบายที่ชัดเจนสำหรับการจัดการข้อมูล
10. **กระบวนการ**: การกำหนด